Import Libraries

In [56]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import urllib.request
import math 
import cv2
import pytesseract

Get all the property addresses

In [97]:
# rightmove main URL
from decimal import ROUND_UP
from math import ceil

### initialise search ### 
URL = "https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=POSTCODE%5E1553684&minBedrooms=2&maxPrice=700000&radius=1.0&index=72&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="
page = requests.get(URL)

# parse search results card
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id='l-container')

# index for iterator (remember each page has 24 ads)
noOfResults = soup.find('div', attrs={'id': 'searchHeader'})
noOfResults = noOfResults.select('span')[0].text
noOfResults = ceil(int(noOfResults)/24) # round up to nearest page

### Start Loop ###
i = 0
all_ids = np.array([])
adLinks = np.array([])

print(noOfResults, " Pages available")

for x in range(noOfResults):
    URL = "https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=POSTCODE%5E1553684&minBedrooms=2&maxPrice=700000&radius=1.0&index=" + str(i) + "&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="
    page = requests.get(URL)

    # parse search results card
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id='l-container')

    # Extract and store ids
    links = results.select('a')
    for card in links:
        ids = card.get('id')
        ids = ids.strip() if ids is not None else ''
        all_ids = np.append(all_ids, ids)

    # clean array and add links
    all_ids = all_ids[all_ids != '']
    cleanIds = np.char.strip(all_ids, 'prop')
    allLinks = np.char.add(cleanIds, '#/?channel=RES_BUY')
    allLinks = np.char.add('https://www.rightmove.co.uk/properties/', allLinks)

    i = i + 24

# removes empty and repeated links 
cleanLinks = np.char.replace(allLinks, 'https://www.rightmove.co.uk/properties/0#/?channel=RES_BUY', '')
cleanLinks = cleanLinks[cleanLinks != '']
cleanLinks = np.unique(cleanLinks)

print('Number of links acquired: ', len(cleanLinks))
print(cleanLinks)


10  Pages available
Number of links acquired:  219
['https://www.rightmove.co.uk/properties/102227564#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/107409182#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/109937678#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/110514809#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/111645674#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/114573920#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/114832574#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/115021484#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/115645691#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116106038#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116106077#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116108690#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116233709#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/propertie

Fishing Properties

In [99]:
i = 0
all_info = np.array([])
data = np.array([])

for property in cleanLinks:
    ad = requests.get(property)
    soup = BeautifulSoup(ad.content, "html.parser")

    # extract property data
    propertyAddress = soup.select('h1')[0].text
    propertyType = soup.select('p')[0].text
    noOfBeds = soup.select('p')[1].text
    noOfBathrooms = soup.select('p')[2].text
    priceLink = soup.body.find('div', attrs={'class': '_1gfnqJ3Vtd1z40MlC0MzXu'})
    price = priceLink.select('span')[0].text
    mapLocation = np.char.add('https://www.rightmove.co.uk/properties/', cleanIds[i])
    floorPlan = np.char.add(mapLocation, '#/floorplan?activePlan=1&channel=RES_BUY')
    mapLocation = np.char.add(mapLocation, '#/map?channel=RES_BUY')
    
    # filter through the images to find the floor plan link
    floorPlanImage = requests.get(floorPlan)
    floorPlanImage = BeautifulSoup(floorPlanImage.content, "html.parser")
    floorPlanImage = floorPlanImage.find('div', attrs={'id': 'root'})
    for j in range(len(floorPlanImage.select('img'))):
        nextFloorPlanImage = floorPlanImage.select('img')[j].get_attribute_list('src',str)
        nextFloorPlanImage = ' '.join(nextFloorPlanImage)
        if '_max_296x197' in nextFloorPlanImage:
            foundFloorPlanImage = nextFloorPlanImage.replace('_max_296x197', '')
            break
        else:
            pass
    
    data = [price, propertyAddress, noOfBeds, noOfBathrooms, foundFloorPlanImage, str(mapLocation)]
    all_info = np.append(all_info, data)
    i = i+1 

all_info = np.split(all_info, len(cleanLinks))
print(all_info)

[array(['£475,000', 'Echo Court, Telegraph Avenue, NW9', 'Apartment', '×3',
       'https://media.rightmove.co.uk/dir/229k/228116/127334546/228116_BEA220786_FLP_00_0000.jpeg',
       'https://www.rightmove.co.uk/properties/127334546#/map?channel=RES_BUY'],
      dtype='<U136'), array(['£320,000', 'Watford Way, Hendon, London, NW4', 'Flat', '×2',
       'https://media.rightmove.co.uk/dir/36k/35948/78041184/35948_3894655_FLP_00_0002.png',
       'https://www.rightmove.co.uk/properties/78041184#/map?channel=RES_BUY'],
      dtype='<U136'), array(['£550,000', 'Sunny Gardens Road, Hendon, NW4', 'Flat', '×4',
       'https://media.rightmove.co.uk/dir/250k/249824/125235137/249824_flat_golders_rise_london_greater_london_nw4_2hr_3S_1_FLP_00_0000.jpeg',
       'https://www.rightmove.co.uk/properties/125235137#/map?channel=RES_BUY'],
      dtype='<U136'), array(['£499,000', 'Hawfinch House, 1 Moorhen Drive, London, NW9',
       'Apartment', '×2',
       'https://media.rightmove.co.uk/dir/12k/1143

In [108]:
# Load the img
req = urllib.request.urlopen(all_info[3][4])
arr = np.asarray(bytearray(req.read()))
img = cv2.imdecode(arr, -1) # 'Load it as it is'
imgScaled = cv2.resize(img, (974, 929))  

# Image Processing
hsv = cv2.cvtColor(imgScaled, cv2.COLOR_BGR2HSV)
msk = cv2.inRange(hsv, np.array([0, 0, 175]), np.array([179, 255, 255]))
krn = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 3))
dlt = cv2.dilate(msk, krn, iterations=1)
thr = 255 - cv2.bitwise_and(dlt, msk)

# Display image
# cv2.imshow('Floor Plan', thr)
# if cv2.waitKey() & 0xff == 27: quit()

# OCR
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
configs = r'--psm 6'
rawText = pytesseract.image_to_string(thr, config=configs)
print(rawText)
findArea = rawText.index('Area = ')
print(rawText[findArea+7:findArea+17])

GROUND FLOOR ASTFLOOR
pr——&: 2-41
yi BATHROOM
CONSERVATORY
'
BEDROOM.
ov
LANDING
KITCHEN
UMNG ROOK
BEOROOMS
| HALLWAY BEDROOM 2
=
‘Yee ates ben rae ese beryl ak coors ETE
‘a tts ent an oe deat ppt epee aE
ste rts une Ths eA state pes oy Powe es Sa
especie gas Tere Ses apes eae len Sed eee
TBescer aparece ot
‘Se wih Mewepa CE?



ValueError: substring not found